# Project : Data Cleaning - Tidy up messy Datasets (Movies Dataset)

From Alexander Hagmann's course "Work with Pandas, SQL Databases, JSON, Web APIs & more to master your real-world Machine Learning & Finance Projects" on Udemy.

## First Steps 

1. __Load__ and __inspect__ the messy dataset __movies_metadata.csv__. Identify columns with nested / stringified json data.

In [488]:
import numpy as np
import pandas as pd
pd.options.display.max_columns=30

In [489]:
data=pd.read_csv("movies_metadata.csv",low_memory=False)

In [490]:
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


- Nested/stringified columns are :  ["belongs_to_collection","genres","production_companies","production_countries","spoken_languages"]

In [491]:
nested_cols = ["belongs_to_collection","genres","production_companies","production_countries","spoken_languages"]

In [492]:
type(data[nested_cols[0]][45464])

float

## Dropping irrelevant Columns

2. __Drop__ the irrelevant columns 'adult', 'imdb_id', 'original_title', 'video' and 'homepage'.

In [493]:
data.adult.value_counts()

False                                                                                                                             45454
True                                                                                                                                  9
 - Written by Ørnås                                                                                                                   1
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
Name: adult, dtype: int64

In [494]:
data.drop(columns=['adult', 'imdb_id', 'original_title', 'video','homepage'],inplace=True)

In [495]:
data_old = data.copy()

## How to handle stringified JSON columns

3. __Evaluate__ Python Expressions in the stringified columns ["belongs_to_collection", "genres", "production_countries", "production_companies", "spoken_languages"] and __remove quotes__ ("") where possible.

In [496]:
import json,ast

In [497]:
data.belongs_to_collection[0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [498]:
json1="{'cat':3,'dog':5}"

In [499]:
# #error
# json.loads(json1)

In [500]:
json2='{"cat":3,"dog":5}'
json.loads(json2)

{'cat': 3, 'dog': 5}

In [501]:
json1

"{'cat':3,'dog':5}"

In [502]:
json1.replace("'",'"')

'{"cat":3,"dog":5}'

In [503]:
json.loads(json1.replace("'",'"'))

{'cat': 3, 'dog': 5}

In [504]:
tst= data.genres.apply(lambda x: json.loads(x.replace("'",'"')) )

In [505]:
tst[0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

**There is a better approach using abstract syntax tree i.e. ast**

In [506]:
ast.literal_eval(json1), ast.literal_eval(json2)

({'cat': 3, 'dog': 5}, {'cat': 3, 'dog': 5})

In [507]:
data.genres.apply(eval)[0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [508]:
data.genres = data.genres.apply(ast.literal_eval)

In [509]:
data.genres[0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [510]:
# #error
# data.belongs_to_collection.apply(ast.literal_eval)

In [511]:
# def unstring_mixdata(value):
#     if isinstance(value,str):
#         return eval(value)
#     else:
#         return np.nan

# data.belongs_to_collection.apply(unstring_mixdata)[0]

In [512]:
nested_cols

['belongs_to_collection',
 'genres',
 'production_companies',
 'production_countries',
 'spoken_languages']

In [513]:
data.belongs_to_collection = data.belongs_to_collection.apply(lambda x : ast.literal_eval(x) if isinstance(x,str) else np.nan )

In [514]:
data.belongs_to_collection[0]

{'id': 10194,
 'name': 'Toy Story Collection',
 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',
 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}

In [515]:
data.production_companies = data.production_companies.apply(lambda x : ast.literal_eval(x) if isinstance(x,str) else np.nan )

In [516]:
data.production_countries = data.production_countries.apply(lambda x : ast.literal_eval(x) if isinstance(x,str) else np.nan )

In [517]:
data.spoken_languages = data.spoken_languages.apply(lambda x : ast.literal_eval(x) if isinstance(x,str) else np.nan )

In [518]:
data.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0


## How to flatten nested Columns

4. __Extract__ only the __collection name__ from the column "belongs_to_collection" and __overwrite__ "belongs_to_collection". <br> For example: The value in the first row (Toy Story) should be 'Toy Story Collection'.

In [519]:
data.belongs_to_collection[0]

{'id': 10194,
 'name': 'Toy Story Collection',
 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',
 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}

In [520]:
data.belongs_to_collection = data.belongs_to_collection.apply(lambda x : x["name"] if isinstance(x,dict) else np.nan)

In [521]:
data.belongs_to_collection.value_counts(dropna=False)

NaN                              40975
The Bowery Boys                     29
Totò Collection                     27
James Bond Collection               26
Zatôichi: The Blind Swordsman       26
                                 ...  
Glass Tiger collection               1
Kathleen Madigan Collection          1
The Big Bottom Box                   1
Joséphine - Saga                     1
Red Lotus Collection                 1
Name: belongs_to_collection, Length: 1696, dtype: int64

5. __Extract__ all __genre names__ from the column "genres" and __overwrite__ "genres". If a movie has more than one genre, __seperate genres by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'Animation|Comedy|Family'.

In [522]:
data.genres

0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                           [{'id': 35, 'name': 'Comedy'}]
                               ...                        
45461    [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...
45462                        [{'id': 18, 'name': 'Drama'}]
45463    [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...
45464                                                   []
45465                                                   []
Name: genres, Length: 45466, dtype: object

In [523]:
data.genres[45464]#["name"]

[]

In [524]:
# def get_genre(lst):
#     names = []
#     for x in lst:
#         names.append(x["name"]) 
#     return "|".join(names)    

In [525]:
# def get_name(lst):
#     names = []
#     try:
#         if(len(lst)!=0):
#             for x in lst:
#                 names.append(x["name"])             
#             return "|".join(names)   
#         else:
#             return np.nan
#     except:
#         #print("The error is ",e)
#         print("The input is ",lst)
#         return np.nan

In [526]:
data.genres[45464]

[]

In [527]:
data.genres = data.genres.apply(lambda x:"|".join(val["name"] for val in x) if isinstance(x,list) else np.nan)

In [528]:
data.genres.value_counts(dropna=False)

Drama                              5000
Comedy                             3621
Documentary                        2723
                                   2442
Drama|Romance                      1301
                                   ... 
Action|Drama|Comedy|Documentary       1
War|Drama|History|Thriller            1
Horror|Drama|History|Thriller         1
Comedy|Crime|Action|Drama             1
Family|Animation|Romance|Comedy       1
Name: genres, Length: 4069, dtype: int64

In [529]:
data.genres.replace("",np.nan,inplace=True)

In [530]:
data.genres.value_counts(dropna=False)

Drama                              5000
Comedy                             3621
Documentary                        2723
NaN                                2442
Drama|Romance                      1301
                                   ... 
Action|Drama|Comedy|Documentary       1
War|Drama|History|Thriller            1
Horror|Drama|History|Thriller         1
Comedy|Crime|Action|Drama             1
Family|Animation|Romance|Comedy       1
Name: genres, Length: 4069, dtype: int64

6. __Extract__ all __spoken language names__ from the column "spoken_languages" and __overwrite__ "spoken_languages". If a movie has more than one spoken language, __seperate spoken languages by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'English'.

In [531]:
data.spoken_languages[0]

[{'iso_639_1': 'en', 'name': 'English'}]

In [532]:
data.spoken_languages = data.spoken_languages.apply(lambda x: "|".join(val["name"] for val in x)
                            if isinstance(x,list) else np.nan )

In [533]:
data.spoken_languages.replace("",np.nan,inplace=True)

In [534]:
data.spoken_languages.value_counts(dropna=False)

English                           22395
NaN                                3958
Français                           1853
日本語                                1289
Italiano                           1218
                                  ...  
Deutsch||ελληνικά|English             1
English|suomi|Deutsch|svenska         1
English|Français|Deutsch|فارسی        1
svenska|English|Español               1
Fulfulde|English                      1
Name: spoken_languages, Length: 1842, dtype: int64

7. __Extract__ all __production countries names__ from the column "production_countries" and __overwrite__ "production_countries". If a movie has more than one production country, __seperate production countries by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'United States of America'.

In [535]:
data.production_countries[0]

[{'iso_3166_1': 'US', 'name': 'United States of America'}]

In [536]:
data.production_countries = (data.production_countries.apply(lambda x: "|".join(i["name"] for i in x) if
                                                            isinstance(x,list) else np.nan))

In [537]:
data.production_countries.replace("",np.nan,inplace=True)

In [538]:
data.production_countries.value_counts(dropna=False)

United States of America                  17851
NaN                                        6288
United Kingdom                             2238
France                                     1654
Japan                                      1356
                                          ...  
Romania|United Kingdom|Canada                 1
Finland|Germany|Netherlands                   1
France|Denmark|Spain|Sweden                   1
France|United States of America|Canada        1
Egypt|Italy|United States of America          1
Name: production_countries, Length: 2390, dtype: int64

8. __Extract__ all __production companies names__ from the column "production_companies" and __overwrite__ "production_companies". If a movie has more than one production company, __seperate production companies by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'Pixar Animation Studios'

In [539]:
data.production_companies[0]

[{'name': 'Pixar Animation Studios', 'id': 3}]

In [540]:
data.production_companies = data.production_companies.apply(lambda x:("|".join(i["name"] for i in x)) if isinstance(x,list) else np.nan)

In [541]:
data.production_companies.replace("",np.nan,inplace=True)

In [542]:
data.production_companies.value_counts(dropna=False)

NaN                                                                                                                                           11881
Metro-Goldwyn-Mayer (MGM)                                                                                                                       742
Warner Bros.                                                                                                                                    540
Paramount Pictures                                                                                                                              505
Twentieth Century Fox Film Corporation                                                                                                          439
                                                                                                                                              ...  
HBO Films|Moving Pictures                                                                                       

In [543]:
data.isna().sum()

belongs_to_collection    40975
budget                       0
genres                    2442
id                           0
original_language           11
overview                   954
popularity                   5
poster_path                386
production_companies     11881
production_countries      6288
release_date                87
revenue                      6
runtime                    263
spoken_languages          3958
status                      87
tagline                  25054
title                        6
vote_average                 6
vote_count                   6
dtype: int64

In [544]:
data_old.isna().sum()

belongs_to_collection    40972
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
vote_average                 6
vote_count                   6
dtype: int64

In [545]:
data.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,Toy Story Collection,30000000,Animation|Comedy|Family,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0
1,NaN,65000000,Adventure|Fantasy|Family,8844,en,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,English|Français,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0


9. __Inspect__ all columns above with value_counts(). Do you see anything strange? __Take reasonable measures__!

## Cleaning Numerical Columns

10. __Convert__ the datatype in the columns __"budget"__, __"id"__ and __"popularity"__ __to numeric__. Set invalid values as NaN.

In [546]:
# def get_budget(val):
#     try:
#         return ast.literal_eval(val)
#     except:
#         return np.nan
# data.budget.apply(get_budget).value_counts()

In [547]:
data.budget =  pd.to_numeric(data.budget,errors='coerce')

In [548]:
data.budget.value_counts(dropna=False)

0.0           36573
5000000.0       286
10000000.0      259
20000000.0      243
2000000.0       242
              ...  
9750000.0         1
7275000.0         1
78146652.0        1
280.0             1
1254040.0         1
Name: budget, Length: 1224, dtype: int64

In [549]:
#data.budget.replace(0,np.nan,inplace=True)

In [550]:
data.id = pd.to_numeric(data.id,errors='coerce')

In [551]:
data.id.value_counts(dropna=False)

NaN         3
141971.0    3
11115.0     2
25541.0     2
15028.0     2
           ..
55135.0     1
15877.0     1
72272.0     1
1549.0      1
461257.0    1
Name: id, Length: 45434, dtype: int64

- There are duplicated ids, meaning duplicated values in the data set.

In [552]:
data.popularity = pd.to_numeric(data.popularity,errors='coerce')

In [553]:
data.popularity.value_counts(dropna=False)

0.000000    66
0.000001    56
0.000308    43
0.000220    40
0.000844    38
            ..
4.762242     1
6.173752     1
0.645652     1
0.556292     1
0.163015     1
Name: popularity, Length: 43758, dtype: int64

11. __Analyze__ the columns __"budget"__ and __"revenue"__ and __"runtime"__. Analyze movies with a budget/revenue/runtime of 0. Do you think the value 0 is the most appropriate value? __Take reasonable measures__! 

In [554]:
data.budget.value_counts(dropna=False)

0.0           36573
5000000.0       286
10000000.0      259
20000000.0      243
2000000.0       242
              ...  
9750000.0         1
7275000.0         1
78146652.0        1
280.0             1
1254040.0         1
Name: budget, Length: 1224, dtype: int64

In [555]:
data.revenue.value_counts()

0.0           38052
12000000.0       20
11000000.0       19
10000000.0       19
2000000.0        18
              ...  
36565280.0        1
439564.0          1
35610100.0        1
10217873.0        1
1413000.0         1
Name: revenue, Length: 6863, dtype: int64

In [556]:
data.runtime.value_counts()

90.0     2556
0.0      1558
100.0    1470
95.0     1412
93.0     1214
         ... 
410.0       1
283.0       1
238.0       1
566.0       1
780.0       1
Name: runtime, Length: 353, dtype: int64

In [557]:
(data[(data.budget==0.0) | (data.runtime==0.0) | (data.revenue == 0.0)]
 [['title','budget',"runtime",'revenue',"popularity","vote_average","vote_count"]])

,title,budget,runtime,revenue,popularity,vote_average,vote_count
2,Grumpier Old Men,0.0,101.0,0.0,11.712900,6.5,92.0
4,Father of the Bride Part II,0.0,106.0,76578911.0,8.387519,5.7,173.0
6,Sabrina,58000000.0,127.0,0.0,6.677277,6.2,141.0
7,Tom and Huck,0.0,97.0,0.0,2.561161,5.4,45.0
11,Dracula: Dead and Loving It,0.0,88.0,0.0,5.430331,5.7,210.0
...,...,...,...,...,...,...,...
45461,Subdue,0.0,90.0,0.0,0.072051,4.0,1.0
45462,Century of Birthing,0.0,360.0,0.0,0.178241,9.0,3.0
45463,Betrayal,0.0,90.0,0.0,0.903007,3.8,6.0
45464,Satan Triumphant,0.0,87.0,0.0,0.003503,0.0,0.0


In [558]:
data.loc[:,["budget","runtime","revenue"]] = data[["budget","runtime","revenue"]].replace(0.0,np.nan)

In [559]:
data["budget"].value_counts(dropna=False).head(5)

NaN           36576
5000000.0       286
10000000.0      259
20000000.0      243
2000000.0       242
Name: budget, dtype: int64

In [560]:
data["revenue"].value_counts(dropna=False).head(5)

NaN           38058
12000000.0       20
11000000.0       19
10000000.0       19
2000000.0        18
Name: revenue, dtype: int64

In [561]:
data["runtime"].value_counts(dropna=False).head(5)

90.0     2556
NaN      1821
100.0    1470
95.0     1412
93.0     1214
Name: runtime, dtype: int64

- Movies with 0 value of budget or revenew or runtime does not make any sense, and also the other features of most of those movies are non-zero. Those values were most likely not availble during the collection, and thus should be replaced by NaN.

12. The columns "budget" and "revenue" shall show values in Million USD. __Convert and Overwrite__!

In [562]:
data.loc[:,["budget","revenue"]] = data[["budget","revenue"]]/1000000  #Converting into million USD 

In [563]:
data.rename(columns={"budget":"budget_musd","revenue":"revenue_musd"},inplace=True)

In [564]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget_musd            8890 non-null   float64
 2   genres                 43024 non-null  object 
 3   id                     45463 non-null  float64
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45460 non-null  float64
 7   poster_path            45080 non-null  object 
 8   production_companies   33585 non-null  object 
 9   production_countries   39178 non-null  object 
 10  release_date           45379 non-null  object 
 11  revenue_musd           7408 non-null   float64
 12  runtime                43645 non-null  float64
 13  spoken_languages       41508 non-null  object 
 14  status                 45379 non-null  object 
 15  ta

13. __Analyze__ movies with a __vote_count of 0__. What´s the __vote_average__ for those movies? Do you think this value is the most appropriate value? __Take reasonable measures__!

In [565]:
data.vote_count.value_counts()

1.0       3264
2.0       3132
0.0       2899
3.0       2787
4.0       2480
          ... 
2755.0       1
1187.0       1
4200.0       1
3322.0       1
2712.0       1
Name: vote_count, Length: 1820, dtype: int64

In [566]:
data.vote_average.value_counts().head(5)

0.0    2998
6.0    2468
5.0    2001
7.0    1886
6.5    1722
Name: vote_average, dtype: int64

In [567]:
(data[(data.vote_count==0.0) & (data.popularity==0.0)]
 [['title','budget_musd',"runtime",'revenue_musd',"popularity","vote_average","vote_count"]]).head()

,title,budget_musd,runtime,revenue_musd,popularity,vote_average,vote_count
5636,Night of the Zombies,NaN,88.0,NaN,0.0,0.0,0.0
13032,My Sons,NaN,120.0,NaN,0.0,0.0,0.0
13839,Bling: A Planet Rock,0.75,90.0,NaN,0.0,0.0,0.0
14890,Lars i porten,NaN,NaN,NaN,0.0,0.0,0.0
16836,Das Experiment 770 - Gebären auf Befehl,NaN,52.0,NaN,0.0,0.0,0.0


- vote_counts can be 0, but vote_avg must be between 1 and 10. Since, we vote_avg is available when vote_count =0, we can replace 0 vote_avg by nan.

In [568]:
data.vote_average.replace(0.0,np.nan,inplace=True)

In [569]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget_musd            8890 non-null   float64
 2   genres                 43024 non-null  object 
 3   id                     45463 non-null  float64
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45460 non-null  float64
 7   poster_path            45080 non-null  object 
 8   production_companies   33585 non-null  object 
 9   production_countries   39178 non-null  object 
 10  release_date           45379 non-null  object 
 11  revenue_musd           7408 non-null   float64
 12  runtime                43645 non-null  float64
 13  spoken_languages       41508 non-null  object 
 14  status                 45379 non-null  object 
 15  ta

## Cleaning DateTime Columns

14. __Convert__ the datatype in the column __"release_date"__ __to datetime__. Set invalid values as NaN.

In [570]:
data.release_date = pd.to_datetime(data.release_date,errors='coerce')

In [571]:
data.release_date.value_counts(dropna=False).head(10)

2008-01-01    136
2009-01-01    121
2007-01-01    118
2005-01-01    111
2006-01-01    101
2002-01-01     96
2004-01-01     90
NaT            90
2001-01-01     84
2003-01-01     76
Name: release_date, dtype: int64

## Cleaning Text / String Columns

15. __Analyze__ the text columns "overview" and "tagline". Try to identify __missing data that is not represented by NaN__ (e.g. "No Data"). __Replace as NaN__ (np.nan)!

In [572]:
data.overview.value_counts(dropna=False).head(5)

NaN                             954
No overview found.              133
No Overview                       7
                                  5
No movie overview available.      3
Name: overview, dtype: int64

In [573]:
[x for x in data.overview.value_counts().keys() if ("No " in x) and (len(x)<40)]

['No overview found.',
 'No Overview',
 'No movie overview available.',
 'No overview yet.',
 'No overview found',
 'No overview',
 'No overview.',
 'No plot overview available']

In [574]:
No_overview_cols = [x for x in data.overview.value_counts().keys() if ("No " in x) and (len(x)<40)]

In [575]:
No_overview_cols.append(" ")

In [576]:
No_overview_cols

['No overview found.',
 'No Overview',
 'No movie overview available.',
 'No overview yet.',
 'No overview found',
 'No overview',
 'No overview.',
 'No plot overview available',
 ' ']

In [577]:
data.overview.replace(No_overview_cols,np.nan,inplace=True)

In [578]:
data.overview.value_counts(dropna=False).head(5)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               1108
A few funny little novels about different aspects of life.                                                                                                                                                                                                                                                                                                                                                                       

In [579]:
data.tagline.value_counts(dropna=False).head(15)

NaN                                                           25054
Based on a true story.                                            7
Trust no one.                                                     4
Be careful what you wish for.                                     4
-                                                                 4
Classic Albums                                                    3
Some doors should never be opened.                                3
A Love Story                                                      3
Drama                                                             3
Know Your Enemy                                                   3
Which one is the first to return - memory or the murderer?        3
How far would you go?                                             3
The end is near.                                                  3
There is no turning back                                          3
There are two sides to every love story.        

In [580]:
data.tagline.value_counts(dropna=False)

NaN                                                           25054
Based on a true story.                                            7
Trust no one.                                                     4
Be careful what you wish for.                                     4
-                                                                 4
                                                              ...  
A special force in a special kind of hell!                        1
Play it. Sing it. Shout it. Feel it.                              1
If It's On TV, It Must Be The Truth.                              1
"I LOVE YOU BABY, BUT MY WIFE JUST REFUSES TO UNDERSTAND!"        1
A deadly game of wits.                                            1
Name: tagline, Length: 20284, dtype: int64

In [581]:
data.tagline.replace('-',np.nan,inplace=True)

In [582]:
data.title.value_counts(dropna=False).iloc[:10]

Cinderella              11
Alice in Wonderland      9
Hamlet                   9
Les Misérables           8
Beauty and the Beast     8
Treasure Island          7
A Christmas Carol        7
The Three Musketeers     7
Blackout                 7
Home                     6
Name: title, dtype: int64

- Title column can have duplicate values. Movies with same name is possible.

In [583]:
data.original_language.value_counts(dropna=False).iloc[:50]

en     32269
fr      2438
it      1529
ja      1350
de      1080
es       994
ru       826
hi       508
ko       444
zh       409
sv       384
pt       316
cn       313
fi       297
nl       248
da       225
pl       219
tr       150
cs       130
el       113
no       106
fa       101
hu       100
ta        78
th        76
he        67
sr        63
ro        57
te        45
ar        39
ml        36
xx        33
bn        29
hr        29
mr        25
is        24
et        24
tl        23
id        20
lv        18
ka        18
sl        17
uk        16
bs        14
ca        12
NaN       11
bg        10
ab        10
vi        10
sk         9
Name: original_language, dtype: int64

## Removing Duplicates

In [584]:
data[data.duplicated(subset=None,keep=False)].sort_values(by="id").head(6)

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
7345,NaN,NaN,Crime|Drama|Thriller,5511.0,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,Fida cinematografica|Compagnie Industrielle et...,France|Italy,1967-10-25,0.039481,105.0,Français,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0
9165,NaN,NaN,Crime|Drama|Thriller,5511.0,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,Fida cinematografica|Compagnie Industrielle et...,France|Italy,1967-10-25,0.039481,105.0,Français,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0
24844,NaN,NaN,Comedy|Drama,11115.0,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,/kHaBqrrozaG7rj6GJg3sUCiM29B.jpg,Andertainment Group|Crescent City Pictures|Tag...,United States of America,2008-01-29,NaN,85.0,English,Released,NaN,Deal,5.2,22.0
14012,NaN,NaN,Comedy|Drama,11115.0,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,/kHaBqrrozaG7rj6GJg3sUCiM29B.jpg,Andertainment Group|Crescent City Pictures|Tag...,United States of America,2008-01-29,NaN,85.0,English,Released,NaN,Deal,5.2,22.0
22151,NaN,NaN,Action|Horror|Science Fiction,18440.0,en,When a comet strikes Earth and kicks up a clou...,1.436085,/tWCyKXHuSrQdLAvNeeVJBnhf1Yv.jpg,NaN,United States of America,2007-01-01,NaN,89.0,English,Released,NaN,Days of Darkness,5.0,5.0
14000,NaN,NaN,Action|Horror|Science Fiction,18440.0,en,When a comet strikes Earth and kicks up a clou...,1.436085,/tWCyKXHuSrQdLAvNeeVJBnhf1Yv.jpg,NaN,United States of America,2007-01-01,NaN,89.0,English,Released,NaN,Days of Darkness,5.0,5.0


16. __Identify__ and __remove__ duplicates!

In [585]:
data.drop_duplicates(keep='first',inplace=True)

In [586]:
data[data.duplicated(subset="id",keep=False)].sort_values(by="id")

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
33826,NaN,30.000000,Comedy|Crime|Drama|Romance|Thriller,4912.0,en,"Television made him famous, but his biggest hi...",7.645827,/o3Im9nPLAgtlw1j2LtpMebAotSe.jpg,Miramax Films|Allied Filmmakers|Mad Chance,United States of America,2002-12-30,33.013805,113.0,English,Released,Some things are better left top secret.,Confessions of a Dangerous Mind,6.6,281.0
5865,NaN,30.000000,Comedy|Crime|Drama|Romance|Thriller,4912.0,en,"Television made him famous, but his biggest hi...",11.331072,/o3Im9nPLAgtlw1j2LtpMebAotSe.jpg,Miramax Films|Allied Filmmakers|Mad Chance,United States of America,2002-12-30,33.013805,113.0,English,Released,Some things are better left top secret.,Confessions of a Dangerous Mind,6.6,281.0
4114,Pokémon Collection,16.000000,Adventure|Fantasy|Animation|Action|Family,10991.0,ja,When Molly Hale's sadness of her father's disa...,10.264597,/5ILjS6XB5deiHop8SXPsYxXWVPE.jpg,TV Tokyo|4 Kids Entertainment|Nintendo|Pikachu...,Japan,2000-07-08,68.411275,93.0,English,Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,143.0
44821,Pokémon Collection,16.000000,Adventure|Fantasy|Animation|Action|Family,10991.0,ja,When Molly Hale's sadness of her father's disa...,6.480376,/5ILjS6XB5deiHop8SXPsYxXWVPE.jpg,TV Tokyo|4 Kids Entertainment|Nintendo|Pikachu...,Japan,2000-07-08,68.411275,93.0,English,Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,144.0
44826,Pokémon Collection,NaN,Adventure|Fantasy|Animation|Science Fiction|Fa...,12600.0,ja,"All your favorite Pokémon characters are back,...",6.080108,/bqL0PVHbQ8Jmw3Njcl38kW0CoeM.jpg,NaN,Japan|United States of America,2001-07-06,28.023563,75.0,日本語,Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,5.7,82.0
5535,Pokémon Collection,NaN,Adventure|Fantasy|Animation|Science Fiction|Fa...,12600.0,ja,"All your favorite Pokémon characters are back,...",7.072301,/bqL0PVHbQ8Jmw3Njcl38kW0CoeM.jpg,NaN,Japan|United States of America,2001-07-06,28.023563,75.0,日本語,Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,5.7,82.0
15765,NaN,0.002500,Drama|Comedy|Foreign,13209.0,fa,"Since women are banned from soccer matches, Ir...",1.529879,/nfkOkpudNNIjRrf0mTFVoiGzHyc.jpg,Jafar Panahi Film Productions,Iran,2006-05-26,NaN,93.0,فارسی,Released,NaN,Offside,6.7,27.0
11342,NaN,0.002500,Drama|Comedy|Foreign,13209.0,fa,"Since women are banned from soccer matches, Ir...",1.528960,/nfkOkpudNNIjRrf0mTFVoiGzHyc.jpg,Jafar Panahi Film Productions,Iran,2006-05-26,NaN,93.0,فارسی,Released,NaN,Offside,6.7,27.0
10419,NaN,1.600000,Drama|Crime|Mystery,14788.0,en,Set against the backdrop of a decaying Midwest...,3.185256,/w56oo9nREcF54sNXVYuE9QxZFjT.jpg,Magnolia Pictures|Extension 765,United States of America,2005-09-03,NaN,73.0,English,Released,NaN,Bubble,6.4,36.0
12066,NaN,1.600000,Drama|Crime|Mystery,14788.0,en,Set against the backdrop of a decaying Midwest...,3.008299,/w56oo9nREcF54sNXVYuE9QxZFjT.jpg,Magnolia Pictures|Extension 765,United States of America,2005-09-03,NaN,73.0,English,Released,NaN,Bubble,6.4,36.0


- The duplicate movies with same id have different popularities. We need to do a much deeper analysis to decide which to keep. For now, we can simply keep the first one following previous convension. 

In [587]:
data.drop_duplicates(subset='id',keep='first',inplace=True,ignore_index=True)

In [588]:
data.columns[:2]

Index(['belongs_to_collection', 'budget_musd'], dtype='object')

In [589]:
#data[data.duplicated(subset="title",keep=False)].title.value_counts()
#data.head()

## Handling Missing Values & Removing Observations

17. __Drop__ all rows/movies with unknown __id__ or __title__.

In [590]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45434 entries, 0 to 45433
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4488 non-null   object        
 1   budget_musd            8880 non-null   float64       
 2   genres                 42992 non-null  object        
 3   id                     45433 non-null  float64       
 4   original_language      45423 non-null  object        
 5   overview               44326 non-null  object        
 6   popularity             45430 non-null  float64       
 7   poster_path            45048 non-null  object        
 8   production_companies   33562 non-null  object        
 9   production_countries   39151 non-null  object        
 10  release_date           45346 non-null  datetime64[ns]
 11  revenue_musd           7398 non-null   float64       
 12  runtime                43615 non-null  float64       
 13  s

In [591]:
data.dropna(axis=0,how='any',subset='id',inplace=True)

In [592]:
data = data.loc[~data.title.isna()]

In [593]:
data[data.title.isna()|data.id.isna()]

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


In [594]:
data.id.astype(int)

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45429    439050
45430    111109
45431     67758
45432    227506
45433    461257
Name: id, Length: 45430, dtype: int32

18. __Keep__ only those rows/movies in the df with __10 or more non-NaN__ values.

In [595]:
data.notna().sum()

belongs_to_collection     4487
budget_musd               8880
genres                   42988
id                       45430
original_language        45419
overview                 44322
popularity               45430
poster_path              45047
production_companies     33562
production_countries     39151
release_date             45346
revenue_musd              7398
runtime                  43615
spoken_languages         41480
status                   45349
tagline                  20397
title                    45430
vote_average             42435
vote_count               45430
dtype: int64

In [596]:
# Number of movies corresponding to number of missing values
data.isna().sum(axis=1).value_counts()

4     12499
3     11437
5      5451
2      4263
1      3859
6      3045
7      1894
0      1132
8      1022
9       515
10      184
11      104
12       21
13        4
dtype: int64

In [597]:
data[data.isna().sum(axis=1)==12]

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
2139,NaN,NaN,NaN,77314.0,fr,NaN,0.037500,/8prmlT6iOYl3zFsDDJl9oDMUGeD.jpg,NaN,NaN,1991-12-04,NaN,NaN,NaN,NaN,NaN,The Cabinet of Dr. Ramirez,NaN,0.0
4129,NaN,NaN,Drama|Thriller|Romance,109472.0,en,NaN,0.001653,NaN,NaN,NaN,2001-06-06,NaN,NaN,NaN,NaN,NaN,The Girl,NaN,0.0
14885,NaN,NaN,NaN,174748.0,no,NaN,0.000000,NaN,NaN,NaN,1984-12-30,NaN,NaN,NaN,Released,NaN,Lars i porten,NaN,0.0
18564,NaN,NaN,Documentary,404471.0,fi,NaN,0.000000,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Released,NaN,Pölynimurikauppiaat,NaN,0.0
19947,NaN,NaN,NaN,397339.0,en,Black and White,0.000000,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Released,NaN,The Awful Truth,NaN,0.0
20293,NaN,NaN,NaN,367678.0,en,American Documentary,0.000000,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Released,NaN,Enola Gay and the Atomic Bombing of Japan,NaN,0.0
22784,NaN,NaN,NaN,158517.0,en,On her way home from an evening shift at a men...,0.000143,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Released,NaN,Lain ulkopuolella,NaN,0.0
24141,NaN,NaN,NaN,287831.0,en,"Harry Raymond, a foreign ambassador in Moscow,...",0.000000,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Released,NaN,External Affairs,NaN,0.0
29290,NaN,NaN,NaN,335141.0,fr,NaN,0.001648,NaN,NaN,NaN,1998-01-01,NaN,NaN,NaN,Released,NaN,Bob le magnifique,NaN,0.0
35627,NaN,NaN,NaN,374698.0,nl,NaN,0.001290,NaN,NaN,NaN,2001-10-24,NaN,NaN,NaN,Released,NaN,Vallen,NaN,0.0


In [598]:
print("If we set allowed number of missing value threshold as 10,\nwe loose ~ {:2.4f}% of movies".format(200/len(data)))

If we set allowed number of missing value threshold as 10,
we loose ~ 0.0044% of movies


In [599]:
data = data.dropna(thresh=10).copy()

In [600]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45117 entries, 0 to 45433
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4487 non-null   object        
 1   budget_musd            8878 non-null   float64       
 2   genres                 42969 non-null  object        
 3   id                     45117 non-null  float64       
 4   original_language      45106 non-null  object        
 5   overview               44138 non-null  object        
 6   popularity             45117 non-null  float64       
 7   poster_path            44885 non-null  object        
 8   production_companies   33561 non-null  object        
 9   production_countries   39147 non-null  object        
 10  release_date           45078 non-null  datetime64[ns]
 11  revenue_musd           7398 non-null   float64       
 12  runtime                43551 non-null  float64       
 13  s

## Final (Cleaning) Steps

19. __Keep__ only those rows/movies in the df with __status "Released"__. Then __drop__ the column "status".

In [601]:
data.status.value_counts(dropna=False)

Released           44690
Rumored              226
Post Production       98
NaN                   66
In Production         20
Planned               15
Canceled               2
Name: status, dtype: int64

In [602]:
data = data.loc[data.status=="Released"].copy()

In [603]:
data.drop(columns='status',inplace=True)

In [604]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44690 entries, 0 to 45433
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4463 non-null   object        
 1   budget_musd            8854 non-null   float64       
 2   genres                 42586 non-null  object        
 3   id                     44690 non-null  float64       
 4   original_language      44680 non-null  object        
 5   overview               43737 non-null  object        
 6   popularity             44690 non-null  float64       
 7   poster_path            44466 non-null  object        
 8   production_companies   33356 non-null  object        
 9   production_countries   38835 non-null  object        
 10  release_date           44657 non-null  datetime64[ns]
 11  revenue_musd           7385 non-null   float64       
 12  runtime                43178 non-null  float64       
 13  s

20. The Order of the columns should be as follows: 

In [605]:
ordered_columns=["id", "title", "tagline", "release_date", "genres", "belongs_to_collection", 
"original_language", "budget_musd", "revenue_musd", "production_companies",
"production_countries", "vote_count", "vote_average", "popularity", "runtime",
"overview", "spoken_languages", "poster_path"]

In [606]:
data = data.loc[:,ordered_columns]

In [607]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44690 entries, 0 to 45433
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     44690 non-null  float64       
 1   title                  44690 non-null  object        
 2   tagline                20284 non-null  object        
 3   release_date           44657 non-null  datetime64[ns]
 4   genres                 42586 non-null  object        
 5   belongs_to_collection  4463 non-null   object        
 6   original_language      44680 non-null  object        
 7   budget_musd            8854 non-null   float64       
 8   revenue_musd           7385 non-null   float64       
 9   production_companies   33356 non-null  object        
 10  production_countries   38835 non-null  object        
 11  vote_count             44690 non-null  float64       
 12  vote_average           41989 non-null  float64       
 13  p

21. __Reset__ the Index and create a __RangeIndex__.

In [610]:
data.reset_index(drop=True,inplace=True)

In [624]:
 data.poster_path =  ("http://image.tmdb.org/t/p/w185"+data.poster_path)

In [626]:
data.head()

,id,title,tagline,release_date,genres,belongs_to_collection,original_language,budget_musd,revenue_musd,production_companies,production_countries,vote_count,vote_average,popularity,runtime,overview,spoken_languages,poster_path
0,862.0,Toy Story,NaN,1995-10-30,Animation|Comedy|Family,Toy Story Collection,en,30.0,373.554033,Pixar Animation Studios,United States of America,5415.0,7.7,21.946943,81.0,"Led by Woody, Andy's toys live happily in his ...",English,http://image.tmdb.org/t/p/w185/rhIRbceoE9lR4ve...
1,8844.0,Jumanji,Roll the dice and unleash the excitement!,1995-12-15,Adventure|Fantasy|Family,NaN,en,65.0,262.797249,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,2413.0,6.9,17.015539,104.0,When siblings Judy and Peter discover an encha...,English|Français,http://image.tmdb.org/t/p/w185/vzmL6fP7aPKNKPR...
2,15602.0,Grumpier Old Men,Still Yelling. Still Fighting. Still Ready for...,1995-12-22,Romance|Comedy,Grumpy Old Men Collection,en,NaN,NaN,Warner Bros.|Lancaster Gate,United States of America,92.0,6.5,11.712900,101.0,A family wedding reignites the ancient feud be...,English,http://image.tmdb.org/t/p/w185/6ksm1sjKMFLbO7U...
3,31357.0,Waiting to Exhale,Friends are the people who let you be yourself...,1995-12-22,Comedy|Drama|Romance,NaN,en,16.0,81.452156,Twentieth Century Fox Film Corporation,United States of America,34.0,6.1,3.859495,127.0,"Cheated on, mistreated and stepped on, the wom...",English,http://image.tmdb.org/t/p/w185/16XOMpEaLWkrcPq...
4,11862.0,Father of the Bride Part II,Just When His World Is Back To Normal... He's ...,1995-02-10,Comedy,Father of the Bride Collection,en,NaN,76.578911,Sandollar Productions|Touchstone Pictures,United States of America,173.0,5.7,8.387519,106.0,Just when George Banks has recovered from his ...,English,http://image.tmdb.org/t/p/w185/e64sOI48hQXyru7...


22. __Save__ the cleaned dataset in a __csv-file__.

In [627]:
data.to_csv("movies_clean.csv",index=False)

In [629]:
pd.read_csv("movies_clean.csv").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44690 entries, 0 to 44689
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     44690 non-null  float64
 1   title                  44690 non-null  object 
 2   tagline                20284 non-null  object 
 3   release_date           44657 non-null  object 
 4   genres                 42586 non-null  object 
 5   belongs_to_collection  4463 non-null   object 
 6   original_language      44680 non-null  object 
 7   budget_musd            8854 non-null   float64
 8   revenue_musd           7385 non-null   float64
 9   production_companies   33356 non-null  object 
 10  production_countries   38835 non-null  object 
 11  vote_count             44690 non-null  float64
 12  vote_average           41989 non-null  float64
 13  popularity             44690 non-null  float64
 14  runtime                43178 non-null  float64
 15  ov